In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from signals import *
aapl = yf.download('AAPL', start='2010-01-01', end='2018-12-31')
adjusted_close_prices = aapl['Adj Close']

/Users/toriwang/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[*********************100%%**********************]  1 of 1 completed


In [2]:
BB_window = 20
rolling_mean = adjusted_close_prices.rolling(window=BB_window).mean()
rolling_std = adjusted_close_prices.rolling(window=BB_window).std()
upper_band = rolling_mean + rolling_std
lower_band = rolling_mean - rolling_std
df = pd.DataFrame(np.array([adjusted_close_prices, upper_band, lower_band]).T, columns=['price', 'upper', 'lower'], index=adjusted_close_prices.index)
df['next_5_ret'] = (df['price'] / df['price'].shift(5) - 1).shift(-5)
df.reset_index(inplace=True)
df.Date = df.Date.dt.strftime('%Y%m%d')

In [3]:
simple_signal(df)
df

AttributeError: module 'ta.trend' has no attribute 'dema_indicator'

In [43]:
df

,Date,price,upper,lower,next_5_ret,MACD,MACD_diff,MACD_signal,RSI,BB_High,BB_Low
0,20100104,6.461977,NaN,NaN,-0.018224,NaN,NaN,NaN,NaN,NaN,NaN
1,20100105,6.473149,NaN,NaN,-0.031066,NaN,NaN,NaN,NaN,NaN,NaN
2,20100106,6.370184,NaN,NaN,-0.001517,NaN,NaN,NaN,NaN,NaN,NaN
3,20100107,6.358409,NaN,NaN,-0.005461,NaN,NaN,NaN,NaN,NaN,NaN
4,20100108,6.400681,NaN,NaN,-0.028540,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2258,20181221,36.073051,42.764921,38.776019,NaN,-2.320779,-0.172095,-2.148684,24.833665,44.658372,36.882569
2259,20181224,35.139694,42.819024,38.112602,NaN,-2.491966,-0.274625,-2.217340,22.983548,45.053066,35.878561
2260,20181226,37.614285,42.670987,37.843022,NaN,-2.400285,-0.146356,-2.253929,36.492543,44.962723,35.551286
2261,20181227,37.370182,42.511789,37.569285,NaN,-2.320574,-0.053316,-2.267258,35.825000,44.857895,35.223180


In [36]:
#df.dropna(axis=0, inplace=True)
df['label_1'] = np.nan
df['label_2'] = np.nan
for i in range(BB_window, len(df)):
    if df.loc[i, 'price'] < df.loc[i, 'lower']:
        df.loc[i, 'label_1'] = 'buy'
    if df.loc[i, 'price'] > df.loc[i, 'upper']:
        df.loc[i, 'label_1'] = 'sell'

for i in range(len(df)-5):
    if df.loc[i, 'next_5_ret'] < -0.02:
        df.loc[i, 'label_2'] = 'sell'
    if df.loc[i, 'next_5_ret'] > 0.02:
        df.loc[i, 'label_2'] = 'buy'
# + return
df

/var/folders/b7/0cfn_y1168g59z4fsf0tqmkm0000gn/T/ipykernel_75781/143346381.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'buy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'label_1'] = 'buy'
/var/folders/b7/0cfn_y1168g59z4fsf0tqmkm0000gn/T/ipykernel_75781/143346381.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sell' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'label_2'] = 'sell'


,Date,price,upper,lower,next_5_ret,label_1,label_2
0,20100104,6.461977,NaN,NaN,-0.018224,NaN,NaN
1,20100105,6.473149,NaN,NaN,-0.031066,NaN,sell
2,20100106,6.370184,NaN,NaN,-0.001517,NaN,NaN
3,20100107,6.358409,NaN,NaN,-0.005461,NaN,NaN
4,20100108,6.400681,NaN,NaN,-0.028540,NaN,sell
...,...,...,...,...,...,...,...
2258,20181221,36.073051,42.764921,38.776019,NaN,buy,NaN
2259,20181224,35.139694,42.819024,38.112602,NaN,buy,NaN
2260,20181226,37.614285,42.670987,37.843022,NaN,buy,NaN
2261,20181227,37.370182,42.511789,37.569285,NaN,buy,NaN


In [37]:
simple_signal(df)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/toriwang/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Close'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/toriwang/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/b7/0cfn_y1168g59z4fsf0tqmkm0000gn/T/ipykernel_75781/1992184277.py", li

In [38]:
df

,Date,price,upper,lower,next_5_ret,label_1,label_2,MACD,MACD_diff,MACD_signal,RSI,BB_High,BB_Low
0,20100104,6.461977,NaN,NaN,-0.018224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20100105,6.473149,NaN,NaN,-0.031066,NaN,sell,NaN,NaN,NaN,NaN,NaN,NaN
2,20100106,6.370184,NaN,NaN,-0.001517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20100107,6.358409,NaN,NaN,-0.005461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20100108,6.400681,NaN,NaN,-0.028540,NaN,sell,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,20181221,36.073051,42.764921,38.776019,NaN,buy,NaN,-2.320779,-0.172095,-2.148684,24.833665,44.658372,36.882569
2259,20181224,35.139694,42.819024,38.112602,NaN,buy,NaN,-2.491966,-0.274625,-2.217340,22.983548,45.053066,35.878561
2260,20181226,37.614285,42.670987,37.843022,NaN,buy,NaN,-2.400285,-0.146356,-2.253929,36.492543,44.962723,35.551286
2261,20181227,37.370182,42.511789,37.569285,NaN,buy,NaN,-2.320574,-0.053316,-2.267258,35.825000,44.857895,35.223180


In [21]:
df1 = df[['Date', 'price', 'upper', 'lower', 'label_1']].copy()
df1.dropna(axis=0, inplace=True)

df2 = df[['Date', 'price', 'upper', 'lower', 'next_5_ret', 'label_2']].copy()
df2.dropna(axis=0, inplace=True)

In [30]:
fig_window_size_list = [20]
fig_width = 0.6
fig_height = 0.64
dpi = 100

for window_size in fig_window_size_list:
    dir_path = f'window_{window_size}_plots'
    os.makedirs(dir_path, exist_ok=True)
    for i in range(window_size, df.shape[0]):
        label1 = df['label_1'].iloc[i]
        label2 = df['label_2'].iloc[i]
        next_5_ret = df['next_5_ret'].iloc[i]
        df_window = df.iloc[i-window_size: i].copy()
        start = df_window.Date.iloc[0]
        end = df_window.Date.iloc[-1]
        plot_path = os.path.join(dir_path, f"window{window_size}_{start}_{end}_{label1}_{label2}_{next_5_ret:.6f}.png")
        plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
        plt.plot(df_window['price'], color='blue')
        plt.plot(df_window['upper'], color='red')
        plt.plot(df_window['lower'], color='green')
        plt.xticks([])
        plt.yticks([])
        plt.savefig(plot_path)
        plt.close()

In [ ]:
def SMA(prices, window=20):
    sma = prices.rolling(window=window).mean()
    return sma

def Bollinger_Bands(prices, std_num=2, window=20):
    sma = SMA(prices, window=window)
    roll_std = prices.rolling(window=window).std()
    upper = sma + std_num * roll_std
    lower = sma - std_num * roll_std
    bb_value = (prices - sma) / (2 * roll_std)
    return upper, lower, bb_value

def Momentum(prices, window=20):
    momentum = prices / prices.shift(window) - 1
    return momentum

def MACD(prices):
    EMA12 = prices.ewm(span=12, adjust=False).mean()
    EMA26 = prices.ewm(span=26, adjust=False).mean()
    macd = EMA12 - EMA26
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def Oscillator(prices, stock_window=14, K_window=3):
    high_stock = prices.rolling(stock_window).max()
    low_stock = prices.rolling(stock_window).min()
    K = (prices - low_stock) / (high_stock - low_stock) * 100
    D = K.rolling(K_window).mean()
    return K, D